

# Sprint22課題 自然言語処理入門

In [1]:
import pandas as pd
import numpy as np
import math

# 正規表現操作のライブラリ
import re

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter, CompoundNounFilter, TokenCountFilter

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

## 【問題1】BoWとN-gram(手計算)

**目的**

- 古典的かつ強力な手法BoWとN-gramの理解  

以下は俳優K.Kさんのつぶやき(コーパス)です。

```
文1: 今撮影中で〜す！
文2: 今休憩中で〜す(^^)
文3: 今日ドラマ撮影で〜す！
文4: 今日、映画瞬公開で〜す！！！
```

**【問】**
特殊文字除去(!や〜など)、単語分割をし以下の2パターンで文1〜文4を数値化(ベクトル化)してください。

- BoW(1-gram)
- BoW(2-gram)

手計算の後見やすい形にしてください。

**例**

```python
import pandas as pd

vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram
```


In [2]:
vocabulary = ["今","今日", "ドラマ", "撮影", "休憩","映画", "瞬","公開", "中", "です"]

ms_kk_texts = [
    "今撮影中で〜す！", 
    "今休憩中で〜す(^^)", 
    "今日ドラマ撮影で〜す！",
    "今日、映画瞬公開で〜す！！！"
]

texts_vec = [
    [1,0,0,1,0,0,0,0,1,1], 
    [1,0,0,0,1,0,0,0,1,1], 
    [0,1,1,1,0,0,0,0,0,1],
    [0,1,0,0,0,1,1,1,0,1]
]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram



,今,今日,ドラマ,撮影,休憩,映画,瞬,公開,中,です
今撮影中で〜す！,1,0,0,1,0,0,0,0,1,1
今休憩中で〜す(^^),1,0,0,0,1,0,0,0,1,1
今日ドラマ撮影で〜す！,0,1,1,1,0,0,0,0,0,1
今日、映画瞬公開で〜す！！！,0,1,0,0,0,1,1,1,0,1


In [3]:
vocabulary = [
    "今撮影", 
    "撮影中", 
    "今休憩", 
    "休憩中", 
    "中です", 
    "今日ドラマ",
    "ドラマ撮影",
    "撮影です", 
    "今日映画", 
    "映画瞬", 
    "瞬公開", 
    "公開です"
]

ms_kk_texts = [
    "今撮影中で〜す！", 
    "今休憩中で〜す(^^)", 
    "今日ドラマ撮影で〜す！",
    "今日、映画瞬公開で〜す！！！"
]

texts_vec = [
    [1,1,0,0,1,0,0,0,0,0,0,0], 
    [0,0,1,1,1,0,0,0,0,0,0,0], 
    [0,0,0,0,0,1,1,1,0,0,0,0],
    [0,1,0,0,0,0,0,0,1,1,1,1]
]

df_bow_2gram = pd.DataFrame(
    data = texts_vec, 
    columns = vocabulary, 
    index = ms_kk_texts
)
df_bow_2gram



,今撮影,撮影中,今休憩,休憩中,中です,今日ドラマ,ドラマ撮影,撮影です,今日映画,映画瞬,瞬公開,公開です
今撮影中で〜す！,1,1,0,0,1,0,0,0,0,0,0,0
今休憩中で〜す(^^),0,0,1,1,1,0,0,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,1,1,1,0,0,0,0
今日、映画瞬公開で〜す！！！,0,1,0,0,0,0,0,0,1,1,1,1


## 【問題2】TF-IDF(手計算)
**目的**

- 古典的かつ強力なTF-IDFの理解

標準的なTF-IDFの公式  
Term Frequency:

$$
tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}
$$

$n_{t,d}$: 文書d内の単語tの出現回数  
$\sum_{s \in d}n_{s,d}$文書dの全単語の出現回数の和  

Inverse Document Frequency:

$$
idf(t) = \log_2{\frac{N}{df(t)}}
$$

$N$ : 全文書数  
$df(t)$: 単語tが出現する文書数

TF-IDF:

$$
tfidf(t, d) = tf(t, d) \times idf(t)
$$

**【問】**  
問題1のコーパスを使って、文1〜文4をTFIDFで数値化(ベクトル化)してください。   
問題1と同様、手計算の後見やすい形にしてください。

*正解例*  
tfidf(今, 文書1) = 0.25 になります。



In [4]:
# 上記文章におけるtfidfの計算
def tfidf(t, d):
    '''
    TF-IDFの計算
    
    Parameter
    ----------
    t : 単語
    d : 文書
    
    Return
    ---------
    TF-IDF
    '''
    # TFの計算
    tf = df_bow_1gram.loc[d ,[t]].values[0]\
                / sum(df_bow_1gram.loc[d])

    # IDFの計算
    idf = np.log2(df_bow_1gram.shape[0]
                  / df_bow_1gram[df_bow_1gram[t]>0].shape[0])
    
    # TF-IDF
    return tf * idf



In [5]:
tfidf('今', '今撮影中で〜す！')

0.25

## 【問題3】テキストクリーニング(プログラミング)
**目的**

- 実データ対応のためのテキストクリーニングの理解
- 正規表現の理解


実際のテキストデータは非常に汚いことが多いです。  
以下は3/6(水)にnoroさんがSlackで発言した文章で、良い例です。

```
<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:
```

**【問】**  
このテキストに以下の処理を施してください。

- urlを削除
- 【〇〇】を削除
- 改行等の特殊文字を削除
- 絵文字除去


ここではしませんが、数字を文字列NUMBERに置き換える処理をよくします。

**正解例**  
```
有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！
```

**正規表現のサンプル**
```python
# 正規表現操作のライブラリ
import re
# 対象テキストデータ
text = '【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\n'
# re.compileを使うと処理が早くなります
BAD_SYMBOL = re.compile('[\n*！`]+')
# re.sub(r'[\n*！`]+', '', text)でもできます
text = re.sub(BAD_SYMBOL, '', text)
text
```

リアルタイムで正規表現を確認できるサイトです。  
https://regex101.com/

https://regex-testdrive.com/ja/dotest

[re 正規表現操作](https://docs.python.org/ja/3/library/re.html)

### Tips NLPのLinuxコマンド
これまでpythonでファイルを読み込んで処理をしていましたが、
簡単な作業においてはlinuxコマンドの方がメモリの使用料が半分以下だったりとパフォーマンスが良いです。

例えばファイルの行数を数えたい場合、pythonでわざわざ書くのは面倒です。
以下の1行のコマンドで実行できます。

```
wc -l 〇〇.txt
```

また分割したい場合はsplit  
並び替えたい場合はsort  
置換にはsed  
文の先頭、後頭部分を見たければhead,tail  
など便利なコマンドがあります。  
詳しく知りたい方はNLP100本ノックで調べてみてください。  

In [6]:
# 対象テキストデータ
text = '<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:'

# 正規表現一覧
symbol_reg = r'[\n*` ]+'  # 改行等の特殊文字を削除
mention_reg = r'<.*?>'  #URL除去
phraze_reg = r'【.*?】'  # 【〇〇】を削除
command_reg = r':[^0-9０-９]+:'  # 絵文字除去

reg_str = f'{symbol_reg}|{mention_reg}|{phraze_reg}|{command_reg}'
reg_str = re.compile(reg_str)

# re.sub(r'[\n*！`]+', '', text)でもできます
text = re.sub(reg_str, '', text)

print(text)


有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！


## 【問題4】形態素解析
**目的**

- 形態素解析の理解  

形態素解析のツールはMecabやJanomeなど様々ですが、  
ここでは手軽に導入できるJanomeを使います。  
[Janome document](https://mocobeta.github.io/janome/)

**【問】**
上記のクリーニングしたテキストをJanomeを用いて形態素解析をし、  
名詞または動詞の単語を抜き出してください。  

**正解例**
```python
["有償", "RubyMine", "Pycharm", ...]
```


In [7]:
t = Tokenizer()
for token in t.tokenize(text):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
ご	接頭詞,名詞接続,*,*,*,*,ご,ゴ,ゴ
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
！	記号,一般,*,*,*,*,！,！,！
この	連体詞,*,*,*,*,*,この,コノ,コノ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
、	記号,読点,*,*,*,*,、,、,、
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
で	助

In [8]:
print([token.surface for token in t.tokenize(text)
      if token.part_of_speech.startswith('名詞,一般')])

['有償', 'RubyMine', 'PyCharm', 'OFF', 'クーポン', 'PyCharm', 'メーカー', 'JetBrains', 'クーポン', 'コード', 'クーポン', 'RubyMinePyCharm', '手', '方式', '手', 'スタンプ', 'クリック', '期限', '月', '水']


In [9]:
print([token.surface for token in t.tokenize(text)
      if token.part_of_speech.startswith('動詞')])

['いたし', '実り', 'いただく', 'でき', '挙げ', 'あげ', 'し', 'ください', 'さ', 'せ', 'いただき', 'し', 'おり']


## 【問題5】ニュースの分析
**目的**

- 日本語の自然言語処理の体験
- 類似度の理解

以下からldcc-20140209.tar.gzをダウンロードしてください。 
[livedoor](https://www.rondhuit.com/download.html#ldcc)

もしくはwgetコマンドを使っても良いです。

```python
# livedoorのnewsをダウンロード
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# 圧縮ファイルを解凍
tar zxf ldcc-20140209.tar.gz
# livedoorニュースの説明を表示
cat text/README.txt
```

**サンプルコード**
```python
# サブフォルダまで賢く読み込んでもらう
from sklearn.datasets import load_files
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data
# 今回はラベルが無いと仮定してください
# targets = bin_data.target
```

【問】
以下の流れでニュースを分析してください。

- まずどんなニュースなのか読んでみる
- 出現単語をカウントして分析する
- テキストをクリーニングする
- BoW + TFIDFでベクトル化する
- あるニュースに一番cos類似度が近いニュースを出力する関数の作成
- 別の類似度手法を1つ調べて上の関数に組み込む(切り替えられるようにする)
- なぜそのような結果になったのか考察する

[sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [10]:
# サブフォルダまで賢く読み込んでもらう
from sklearn.datasets import load_files
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data
# 今回はラベルが無いと仮定してください
# targets = bin_data.target

#### まずどんなニュースなのか読んでみる

In [11]:
# データサイズが大きいため、一部抜粋。
documents[0:10]

['http://news.livedoor.com/article/detail/4931238/\n2010-08-08T10:00:00+0900\nNY名物イベントが日本でも！名店グルメを気軽に楽しむ\nニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\n\u3000\n\u3000実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。\n\n参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。\n\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\n\n\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？\n\nJAPAN RESTAURANT WEEK 2010 -公式サイト\n',
 'http://news.livedoor.com/article/detail/6655079/\n2012-06-13T19:25:00+0

#### 出現単語をカウントして分析する

In [12]:
token_count = [POSKeepFilter('名詞,一般'), TokenCountFilter(sorted=True)]
a = Analyzer(token_filters=token_count)
for k,v in a.analyze(''.join(documents[0:10])):
    print('%s: %d'%(k, v))

/: 45
.: 28
こと: 26
-: 25
さん: 23
の: 23
認識: 23
:: 20
00: 17
発音: 17
人: 16
livedoor: 14
年: 14
日: 14
機能: 14
通: 14
プレゼント: 13
音声: 13
練習: 12
英会話: 12
http: 11
://: 11
+: 11
2: 11
1: 11
あなた: 11
会社: 11
news: 10
com: 10
article: 10
detail: 10
T: 10
0900: 10
者: 10
よう: 10
2010: 9
月: 9
小沢: 9
情報: 9
里: 9
田: 9
英語: 9
日本: 8
仲間: 8
亀梨: 8
女性: 8
朝: 8
デジ: 8
アプリ: 8
学習: 8
レストラン: 7
版: 7
ため: 7
氏: 7
3: 7
デジタル: 7
ん: 7
皆さん: 7
的: 7
5: 7
Siri: 7
Real: 7
08: 6
2012: 6
交際: 6
もの: 6
女: 6
途上: 6
国: 6
品: 6
被害: 6
彼: 6
位: 6
転職: 6
化粧: 6
30: 5
中: 5
東京: 5
田中: 5
記事: 5
11: 5
芸能: 5
被災: 5
地: 5
政府: 5
風評: 5
クリスマス: 5
悩み: 5
そう: 5
4: 5
機器: 5
今回: 5
iPhone: 5
活用: 5
日本語: 5
10: 4
イベント: 4
期間: 4
円: 4
三: 4
一郎: 4
支援: 4
離婚: 4
ネット: 4
掲示板: 4
内容: 4
声: 4
関連: 4
12: 4
世界: 4
缶詰: 4
美容: 4
家電: 4
Panasonic: 4
Beauty: 4
キレイ: 4
キャンペーン: 4
ラム: 4
ダッシュ: 4
人気: 4
現在: 4
活躍: 4
環境: 4
提供: 4
フラワー: 4
夏: 4
さ: 4
digi: 4
アクセント: 4
ウィーク: 3
特別: 3
7: 3
31: 3
別: 3
服部: 3
田村: 3
たち: 3
参加: 3
代表: 3
サイト: 3
週刊文春: 3
手紙: 3
夫人: 3
本日: 3
週刊: 3
アサヒ: 3
関係: 3
紹介: 3
コメント: 3
確か: 3
昨年: 3
時: 3
高校: 3

#### テキストをクリーニングする

In [13]:
# 正規表現一覧
symbol_reg = r'[\n*` ]+'  # 改行等の特殊文字を削除
url_reg = r'(https?|ftp)(:\/\/[-_.!~*\'()a-zA-Z0-9;\/?:\@&=+\$,%#]+)'  # URL削除

reg_str = f'{symbol_reg}|{url_reg}'
reg_str = re.compile(reg_str)

# re.sub(r'[\n*！`]+', '', text)でもできます
document_clean = []
for i in range(10):
    document_clean.append(re.sub(reg_str, '', ''.join(documents[i]))) 

print(document_clean)

['2010-08-08T10:00:00+0900NY名物イベントが日本でも！名店グルメを気軽に楽しむニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\u3000\u3000実行委員長には、学校法人服部学園、服部栄養専門学校理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LABETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストランタテルヨシノ汐留」や、日本料理の名門「つきぢ田村」、「金田中庵」、「赤坂璃宮」に「mikuniMARUNOUCHI」など、日本を代表するレストランがずらり。\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？JAPANRESTAURANTWEEK2010-公式サイト', '2012-06-13T19:25:00+0900小沢一郎氏の妻が支援者に離婚を報告。「週刊文春」報じる13日、Web版「週刊文春」は、民主党の元代表・小沢一郎氏の妻が、支援者宛に離婚したことを伝える手紙を送ったと報じ、ツイッターやネット掲示板で大きな話題になっている。記事によると、その手紙は「小沢は放射能が怖くて

#### BoW + TFIDFでベクトル化する

In [14]:
a = Analyzer(token_filters=[CompoundNounFilter(), POSKeepFilter("名詞,一般")])

docs = []
for i in range(10):
    docs.append(" ".join([tok.surface for tok in a.analyze(document_clean[i])]))

In [15]:
#ベクトル化
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

In [16]:
# 各文書で重要な単語上位10語を表示する
feature_names = np.array(vectorizer.get_feature_names())
feature_words_list = np.empty([0,100])
for vec in X:
    index = np.argsort(vec.toarray(), axis=1)[:,::-1]
    feature_words = feature_names[index]
    print(feature_words[:,:10])
    feature_words_list = np.vstack((feature_words_list, feature_words[0][:100]))

[['レストラン' 'ウィーク2010' 'イベント' 'ジャパンレストラン' '参加' '気軽' 'サ別' '日本' '予約方法'
  '服部幸應氏']]
[['離婚' '手紙' '週刊文春' '秘書' '小沢' 'ツイッター' '支援者' '和子夫人' '小沢氏' 'ネット掲示板']]
[['里田' '交際' '週刊アサヒ芸能' '周囲' '高校時代' '田中' '芸能デスク' 'アプローチ' '同誌' 'コメント']]
[['途上国' '被災地' '風評被害' '政府' '缶詰' '製造' '発表' '懸念' 'msn産経ニュース' '非難']]
[['プレゼント' '亀梨さん' '仲間さん' 'クリスマス' 'panasonicbeauty' 'キレイ' '美容家電' 'ドラマ'
  'ラムダッシュ' '亀梨和也さん']]
[['会社' '横行' '環境' '転職' '三年' '悩み' '活躍' '女性' '不倫' 'せい']]
[['ボーダー' '東京ガールズコレクション' 'tgc' '開催' 'コーディネート' 'マリンスタイル' 'そう' 'トレンド'
  'ショートパンツ' 'スウィート']]
[['化粧品' 'あなた' 'こと' 'ライフスタイル' 'さ度' 'どれ' '特徴' 'とんでも' '化粧' '母性本能']]
[['デジ通' 'ゲリラ豪雨' 'オススメ' 'スマホ' 'デジタル機器' 'digi2' '皆さん' '12t10' '関連記事' '空間線量']]
[['英語' '音声認識機能' 'siri' '認識' '発音' '発音練習' '学習' 'アプリ' '音声認識' '日本語アクセント']]


#### あるニュースに一番cos類似度が近いニュースを出力する関数の作成

In [17]:
docs2 = []
for i in range(10):
    docs2.append([tok.surface for tok in a.analyze(document_clean[i])])

In [18]:
def calc_cos(dictA, dictB):
    """
    cos類似度を計算する関数
    @param dictA 1つ目の文章
    @param dictB 2つ目の文章
    @return cos類似度を計算した結果。0〜1で1に近ければ類似度が高い。
    """
    # 文書Aのベクトル長を計算
    lengthA = 0.0
    for key,value in dictA.items():
        lengthA = lengthA + value*value
    lengthA = math.sqrt(lengthA)

    # 文書Bのベクトル長を計算
    lengthB = 0.0
    for key,value in dictB.items():
        lengthB = lengthB + value*value
    lengthB = math.sqrt(lengthB)

    # AとBの内積を計算
    dotProduct = 0.0
    for keyA,valueA in dictA.items():
        for keyB,valueB in dictB.items():
            if keyA==keyB:
                dotProduct = dotProduct + valueA*valueB
    # cos類似度を計算
    cos = dotProduct / (lengthA*lengthB)
    return cos


def words_to_freqdict(words):
    """
    単語の配列を、単語と頻度の辞書に変換する関数
    例: ["X","X","Y","Z","X"] => {"X":3, "Y":1, "Z":1}
    @param words 単語の配列
    @return 単語と頻度の辞書
    """
    freqdict = {}
    for word in words:
        if word in freqdict:
            freqdict[word] = freqdict[word] + 1
        else:
            freqdict[word] = 1
    return freqdict


- 文書0をその他の文書を比較する

In [19]:
document_clean[0]

'2010-08-08T10:00:00+0900NY名物イベントが日本でも！名店グルメを気軽に楽しむニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\u3000\u3000実行委員長には、学校法人服部学園、服部栄養専門学校理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LABETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストランタテルヨシノ汐留」や、日本料理の名門「つきぢ田村」、「金田中庵」、「赤坂璃宮」に「mikuniMARUNOUCHI」など、日本を代表するレストランがずらり。\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？JAPANRESTAURANTWEEK2010-公式サイト'

In [20]:
freqdict0 = words_to_freqdict(docs2[0]) 
for i in range(9):
    freqdict_i = words_to_freqdict(docs2[i+1]) 
    
    cos0_i = calc_cos(freqdict0,freqdict_i)
    
    print('文書0と文書{}のcos類似度 : {:.3f}'.format(i+1, cos0_i)) 

文書0と文書1のcos類似度 : 0.025
文書0と文書2のcos類似度 : 0.006
文書0と文書3のcos類似度 : 0.043
文書0と文書4のcos類似度 : 0.019
文書0と文書5のcos類似度 : 0.009
文書0と文書6のcos類似度 : 0.024
文書0と文書7のcos類似度 : 0.047
文書0と文書8のcos類似度 : 0.009
文書0と文書9のcos類似度 : 0.042


わずかに文書0と文書7のcos類似度が高くなった

In [21]:
document_clean[7]

'2012-07-07T17:00:00+0900あなたの打たれ強さ度は?／イイ女を作る朝型生活など−【ライフスタイル】週間ランキング人間関係をスムーズにするヒントやライフハック、節約ネタなど、今すぐ役立つ情報が詰まったPeachyの「ライフスタイル」カテゴリ。このカテゴリのなかから、2012年6月27日〜7月4日の間に最も多く読まれた記事TOP5をご紹介します！第1位：1年間洗顔せずお化粧を続けるとこうなる!?365日分のお化粧を施したらドロドロになってしもうたよあなたはこんなことを考えたことがありますか？「私って、1年にどれくらい化粧品を使っているんだろう？」そんな素朴な疑問を、モデルを使って実際に目に見える形にしてしまった、2人のオランダ人アーティストがいました。彼らの名は、Lernert&Sander（ふたりとも男性です！）。ただしこのふたりが表現したかったことは、「人は1年に化粧品をどれくらい使うのか」ではなく、「どれくらい化粧品を使えば、人は自然な状態からとんでもない状態になるのか」ということ。彼らは「そうだ！\u3000化粧品を1年分塗り重ねていけば、きっととんでもなくなるはずだ！」と、考えたわけです。第2位：あなたの「打たれ強さ度」を診断恋がうまくいかなかったり、理不尽なことで上司に怒られたり。生きていると凹むことっていっぱいありますよね。どん底気分のとき、あなたはすぐに回復できる人？\u3000それともなかなか立ち直れないほう？\u3000恋愛カウンセラー・ゆまさん監修の「究極の恋愛科学」では、あなたの「打たれ強さ度」をチェックできる心理テストを公開しています。第3位：レパートリーに悩むママ必見！夏のカンタン朝ごはんとは？あなたは毎日朝ごはんを食べていますか？\u3000何かと慌しい朝。「時間がなくて毎日同じメニュー」なんて方も多いのではないでしょうか。ひとりだと適当にパンやおにぎりをコンビニで購入したり、ダイエットのために朝食を抜くこともあるかもしれませんが、家庭を持っているとそうはいきませんよね。第4位：いい女は朝を制す！朝型女子7の習慣「おはようございます」という挨拶からも気品が漂ってきそうな朝からシャキっと美しい女性、あなたの周りにいませんか？今回はそんな女性たちが実践しているであろう、いい女に欠かせない朝の習慣をまとめてみました。第5位：

#### 別の類似度手法を1つ調べて上の関数に組み込む(切り替えられるようにする)
- Doc2Vec

In [22]:
#import gensim

def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    word = []
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
        #名詞だけで比較
        if part_of_speech == "名詞":
            word.append(token.surface)
        '''
        if part_of_speech == "動詞":
            word.append(token.base_form)
        if part_of_speech == "形容詞":
            word.append(token.base_form)
        if part_of_speech == "形容動詞":
            word.append(token.base_form)
        '''
    return word


In [23]:
training_docs = []

for i in range(10):

    words=tokenize(document_clean[i])

    sent = TaggedDocument(words=words ,  tags=[i])

    training_docs.append(sent)

model = Doc2Vec(documents=training_docs, dm=1,
                vector_size=300, window=8, min_count=1, workers=4)

model.train(training_docs, total_examples=model.corpus_count, epochs=50)

#cos類似度
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


for i in range(9):
    ret =cos_sim(model.docvecs[0], model.docvecs[i+1])
    print('文書0と文書{}の類似度 : {:.3f}'.format(i+1, ret)) 

文書0と文書1の類似度 : 0.999
文書0と文書2の類似度 : 0.999
文書0と文書3の類似度 : 0.998
文書0と文書4の類似度 : 0.997
文書0と文書5の類似度 : 0.999
文書0と文書6の類似度 : 0.999
文書0と文書7の類似度 : 0.999
文書0と文書8の類似度 : 0.998
文書0と文書9の類似度 : 0.936


全体として類似度が高く、その中でも文書1,2,5,6との類似度が高くなった

In [24]:
document_clean[2]

'2010-11-30T08:00:00+0900【SportsWatch】田中＆里田の交際、アプローチは里田からグラビアアイドル・ほしのあき＆騎手・三浦皇成がお互いのブログで交際を認めた日、東北楽天ゴールデンイーグルスのエース・田中将大とタレント・里田まいの交際もまた公のものとなったが、本日30日（火）発売の「週刊アサヒ芸能」（12.9号）では、「マー君を“ナンパ”した里田」との見出しで、両者の交際にまつわる関係者の証言を紹介した。同誌にコメントを寄せた芸能デスクによると、「周囲の事情はさておき、当人たちが盛り上がっているのは確か。結婚も完全に視野に入れているようで、すでに新居を探しているとの話まである」という。また、二人の交際は里田からのアプローチによるものとのことで、前出の芸能デスクは、「2人は昨年末の番組共演時に、里田から猛アタック。『一緒に食事でも』と誘い、春頃にはつきあい始めた。里田は仲のいいスザンヌ（24）、木下優樹菜（22）には、早くから相談していたし、周囲にも浮かれてしゃべりまくっていた。次世代エースをゲットしたわけですから、賞味期限切れ間近の里田にすれば、してやったりでしょう」とも語っている。ちなみに、同じく同誌にコメントする球団関係者は、「高校時代は同じ高校の女子生徒と在学中の3年間、ずっとつきあっていましたね。基本はオクテ。プロ入り後にキャバクラなども覚えましたが、まぁ、モテるタイプではないし、女あしらいも不慣れ。高校時代はかなりワガママで、元カノは苦労が絶えなかったと聞いています。同年代の女より、里田のような姉さんタイプのほうが勝負の世界では向いているケースも多いですからね」と明かしており、この交際も田中にとっては“吉”とした。・週刊アサヒ芸能［ライト版］＜デジタル＞（PC版）・週刊アサヒ芸能（モバイル版）'

#### なぜそのような結果になったのか考察する
- 今回使用した文書の中では人の目から見ても特段関連性は考えにくい
- そのため、cos類似度で低い値を出したのは理解できるが、doc2vecで高い値が返されたのは理解しがたい
- 最初の日付構成が一致している等により、類似度が高くなった可能性もある

## 【問題6】感情分析
**目的**

- NLP定番の感情分析の経験
- 英語の処理の実践

以下からLarge Movie Review Datasetをダウンロードしてください。
[IMDBレビュー](http://ai.stanford.edu/~amaas/data/sentiment/)

同じようにwgetコマンドでも可能です。


In [25]:
# サブフォルダまで自動で読み込んでもらう
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_text, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target

### 【問】
IMDBという映画に対するレビューのデータセットを使います。
良いレビューか悪いレビューかを判定するモデルを作ってください。
テストデータに対する正解率が90%を超えるまで、調査=>実行=>改善を繰り返してください。
前処理になぜその処理をしたのかを書くとエンジニアリングとしても完璧です。  
**注意**: 必ず間違っていたデータを観察してください。

In [26]:
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(train_text)
reviews_test_clean = preprocess_reviews(test_text)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(
    X, train_y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    

/Users/yusuke-saruya/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/yusuke-saruya/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.87168
Accuracy for C=0.05: 0.87856
Accuracy for C=0.25: 0.87776
Accuracy for C=0.5: 0.8752
Accuracy for C=1: 0.8728


In [29]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X, train_y)
print ("Final Accuracy: %s" 
       % accuracy_score(test_y, final_model.predict(X_test)))

Final Accuracy: 0.88152


### 間違っているデータの観察

#### 悪いレビューだが、良い(1)と予測されたレビュー

In [30]:
#対象インデックス確認
np.where(([test_y!=final_model.predict(X_test)]) & (test_y==0))[1][:10]

array([11, 25, 32, 34, 57, 65, 70, 71, 73, 91])

In [31]:
print('y_test : {}'.format(test_y[11]))
print('y_predict : {}'.format(final_model.predict(X_test)[11]))
print('predict_prob : {:.3f}'.format(final_model.predict_proba(X_test)[:,1][11]))
print()
print(test_text[11])

y_test : 0
y_predict : 1
predict_prob : 0.786

Even Disney are guilty of the cash cow disease, after the roaring success of The Love Bug in 1968, the house of mouse cashed in with Herbie Rides Again, Herbie Goes To Monte Carlo, and Herbie Goes Bananas. Neither sequel capturing the charm and inoffensive appeal of The Love Bug back in 68, in this one we find race driver Jim Douglas and his sidekick Wheely Applegate, entering Herbie in the Monte Carlo Rally. Naturally things outside of the race start to take over priorities, they get mixed up in a diamond robbery and Herbie falls in love with another car!. The car stunts are of course pleasant and easy on the eye, and it would be churlish of me to really vent venom on such a friendly piece of fluff, it's just that the film goes nowhere fast and personally now i can see it for the coin motivated piece of work it is. Still you get to see Herbie take a bath, foil the baddies and of course dance for the lady in his life, so something there fo

- レビューというより、映画の要約であり、人が読んでも良いか悪いか判断がし辛い。

In [32]:
print('y_test : {}'.format(test_y[25]))
print('y_predict : {}'.format(final_model.predict(X_test)[25]))
print('predict_prob : {:.3f}'.format(final_model.predict_proba(X_test)[:,1][25]))
print()
test_text[25]

y_test : 0
y_predict : 1
predict_prob : 0.806



"While The Twilight Zone was a wonderful show, it was also very uneven--with some great episodes, some lousy ones and many in between. Don't believe the die-hard fans--there were some stinkers and this was definitely one of them.<br /><br />In a plot that is obviously meant to be an attack on Fidel Castro, a near lookalike (Peter Falk in lots of makeup and a beard) obtains a magic mirror that allows him to realize who all his enemies are so he can liquidate them. While I do believe that Castro is a thug and dictator (and tens of thousands of refugees and political prisoners will attest to this), it's amazing how this sort of preachy episode actually makes audiences laugh at the American efforts to marginalize the creep and actually makes Castro seem okay!! Think about it--Serling and company wanted to hurt Castro but instead only seemed to be obvious, preachy and silly in the process.<br /><br />It's indeed bad--almost laughably bad when seen today."

- 冒頭に一部分だけ良い評価をしている
- 直接的に悪いワードが少なく、良いレビューとして予測された？

#### 良いレビューだが、悪い(0)と予測されたレビュー

In [33]:
#対象インデックス確認
np.where(([test_y!=final_model.predict(X_test)]) & (test_y==1))[1][:10]

array([ 16,  37,  43,  56,  68,  79,  93, 131, 132, 147])

In [34]:
print('y_test : {}'.format(test_y[16]))
print('y_predict : {}'.format(final_model.predict(X_test)[16]))
print('predict_prob : {:.3f}'.format(final_model.predict_proba(X_test)[:,1][16]))
print()
test_text[16]

y_test : 1
y_predict : 0
predict_prob : 0.478



'The great James Cagney, top-billed in big letters, doesn\'t show up till the movie\'s second third, and probably has less screen time than Dudley Digges, who plays the eee-vill reform-school potentate. But when Jimmy arrives, as a deputy commissioner of something-or-other out to reform reform schools, he slashes the air with his hands and jumps on the balls of his feet and spits out punchy Warners-First National dialogue with all the customary, and expected, panache. The psychology in this crisp antique, one of Warners\' many efforts to assert its place as the "socially conscious" studio, doesn\'t run deep: Digges is bad just because the script requires him to be, and there\'s the quaint notion that juvenile delinquents will turn into swell kids if they\'re just given a dash of autonomy. But it\'s made in that spare, fast style that the studio specialized in, and it never bores. Frankie Darro, who got into all kinds of onscreen trouble during a brief tenure as Warners\' favorite Rotte

- bores(退屈な)が悪いレビュー結果に影響したか？

In [35]:
print('y_test : {}'.format(test_y[37]))
print('y_predict : {}'.format(final_model.predict(X_test)[37]))
print('predict_prob : {:.3f}'.format(final_model.predict_proba(X_test)[:,1][37]))
print()
test_text[37]

y_test : 1
y_predict : 0
predict_prob : 0.193



'You have to be awfully patient to sit through a film with one-liners so flat and unfunny that you wonder what all the fuss was about when WHISTLING IN THE DARK opened to such an enthusiastic greeting from audiences in the 1940s.<br /><br />On top of some weak one-liners and ordinary sight gags, the plot is as far-fetched as the tales The Fox (Red Skelton) tells his radio audience. You have to wonder why anyone would think he could come up with a real-life solution on how to commit the perfect crime and get away with it. But then, that\'s how unrealistic the comedy is.<br /><br />But--if you\'re a true Red Skelton fan and enjoy a look back at how comedies were made in the \'40s--you can at least enjoy the amiable cast supporting him. Ann Rutherford and Virginia Grey do nicely as his love interest and Conrad Veidt, as always, makes an interesting villain. One of his more amusing moments is his reaction to Skelton explaining the mysteries of wearing turbans. "I never knew that," he muses

- 文章前半部分に、映画背景を知らなければよく理解できないでしょう的な内容があり、悪い結果に影響した？

## 【問題7】分散表現(アドバンス)
**目的**

- 主流である分散表現の理解

分散表現の詳細はこちらを参考にしてください

**【問】**
以下の中から一つ選んで分散表現を獲得し、
好きな単語群をt-SNE,PCAなどを用いて可視化してください。
コーパスは自由です。

- Word2Vec-CBoW(2〜3人)
- Word2Vec-skip-gram(2〜3人)
- fastText(2〜3人)

また以下の4点についてもノートに書いてください。

分布仮説とは何か？
分散表現を得ることのメリットは何か？
上で選んだモデルのメリット、デメリットは何か？
なぜそのパラメータを選んだのか？

**※省略**

## 【問題8】自然言語処理の応用事例
**目的**

- NLPの情報共有  
現在自然言語処理はどのような企業でどのように活用されているか？   
1つ例をあげて3~5分で発表してください。   
(例)メルカリは商品説明をTF-IDFを用いてベクトル化して商品の異常検知を行っている。

### LegalForce
- 2019年4月2日に正式版リリースとなった契約書自動レビューソフトウェア
- １秒で不利な条文や欠落条項を指摘
- 類似契約書のレコメンド
- 自社法務部門チェックの時間が大幅に軽減される
- 前職の営業時代にあったらとても嬉しいと思う。法務部門の契約書チェックに時間がかかり、とても苦労した